# Topic modeling : LDA

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Dataset : Grand Débat National (Great national debate)

The aim of this exercise is to be familiar with the text-mining and topic models such as LDA. One of the contexts where topic modeling is very useful is in open-ended questions. It allows us to explore the variation of topics addressed in people's responses. For this we will use the french "Grand Débat National" dataset. This dataset presents a complete set of responses from the [Grand Débat National](https://granddebat.fr/), the public debate organized by President Macron. The purpose of the debate was to better understand the needs and opinions of the French people following the Yellow Vests protests. The results of this debate are now available as [open data](https://granddebat.fr/pages/donnees-ouvertes).

## 1. Import data

**Question 1 :** Download one of the ecological transition csv files and load the content into a pandas dataframe. Name this variable `raw_data`

In [ ]:
import pandas as pd

# TODO

We will focus on the last question: ``Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?`` We hope that our LDA model will help us to analyze the topics on which their responses are focused. Let's take a look on the data :

In [3]:
question = "Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?"
raw_data[question].head(10)

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
2                                                  NaN
3                                                  NaN
4      Une vrai politique écologique et non économique
5    Les bonnes idées ne grandissent que par le par...
6    Pédagogie dans ce sens là dés la petite école ...
7                                                  NaN
8    faire de l'écologie incitative et non punitive...
9    Développer le ferroutage pour les poids lourds...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

As we note, there is a lot of missing data (like any open-ended question, people decide whether or not to write a comment). A cleanup step is necessary.

## 2. Clean and vectorize documents

Before training our LDA model, we need to tokenize our text. We will tokenize with the [spaCy]  (https://spacy.io/) library because we will only perform some basic preprocessing. We will just initialize a blank template for the French language.

In [4]:
import spacy

nlp = spacy.blank("fr")

Let's remove all the rows from the dataframe that don't have an answer for our question (the `NaNs above). This new dataframe will be called ``texts``.

In [6]:
texts = # TODO

First preprocessing with spacy :

In [9]:
spacy_docs = list(nlp.pipe(texts))

We now have a list of spaCy documents. We will transform each spaCy document into a list of tokens. Instead of the original tokens, we will work with lemmas instead. This will allow our model to generalize better

Here is the full list of preprocessing used: 
 
- remove all **words less than 3 characters**,
- remove all **stop-words**, and
- lemmatize** the remaining words and,
- put these words in **minuscule**.

In [12]:
docs = [[t.lemma_.lower() for t in doc if len(t.orth_) > 3 and not t.is_stop] for doc in spacy_docs]
print(docs[:3])

[['multiplier', 'centrales', 'géothermiques'], ['problèmes', 'trouve', 'confronté', 'ensemble', 'planète', 'dénoncent', 'parfait', 'désordre', 'gilets', 'jaunes', 'france', '-ils', 'surpopulation', 'mondiale', 'population', 'passée', 'd’1,5', 'milliards', 'habitants', '1900', 'milliards', '2020', 'montera', 'bientôt', 'milliards', '2040', 'progrès', 'communication', 'village', 'mondial', 'individu', 'fond', 'asie', 'fond', 'afrique', 'passant', 'quartiers', 'campagnes', 'pays', 'aspire', 'vivre', 'blâmer', 'lotis', 'concitoyens', 'logement', 'nourriture', 'biens', 'consommation', 'déplacement', 'mère', 'problèmes', 'solution', 'problèmes', 'stabilisation', 'croissance', 'démographique', 'partage', 'richesses', 'partage', 'terres', 'partage', 'protection', 'biodiversité', 'règlement', 'conflits', 'lutte', 'déforestation', 'lutte', 'dérèglement', 'climatique', 'règlement', 'conflits', 'stabilisation', 'migrations', 'concurrence', 'commerciale', 'mondiale', 'etc.', 'française', 'européenn

In order to preserve some word order in our modeling, we will take into account frequent bigrams. For this, we will use the [Gensim](https://radimrehurek.com/gensim/)library. We would like to point out that the Gensim library is an excellent NLP library for topics modeling. 

Here is the chosen process: 

- We first identify the frequent bigrams in the corpus, 
- then we add them to the list of tokens for the documents in which they appear. This means that the bigrams will not be in their correct position in the text, but this is not a problem: topic models are bag-of-words models that ignore the position of words anyway.

In [13]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            docs[idx].append(token)

Let's move on to the last steps of the Gensim specific preprocessing. First, we will create a dictionary representation of the documents. This dictionary will map each word to a unique identifier and will help us create word-sack representations of each document. These bag-of-words representations contain the identifiers of the words in the document as well as their frequency. In addition, we can remove the least frequent and most frequent words from the vocabulary. This will improve the quality of our model and speed up its training. The minimum frequency of a word is expressed as an absolute number, the maximum frequency is the proportion of documents in which a word can appear.

In [14]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in original documents :', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Number of unique words after removing rare and common words :', len(dictionary))

print("Example representation of document 3 :", dictionary.doc2bow(docs[2]))

Number of unique words in original documents : 43522
Number of unique words after removing rare and common words : 17781
Example representation of document 3 : [(86, 1), (87, 1), (88, 1), (89, 1)]


Next, we create bag-of-words representations for each document in the corpus see method [doc2bow](https://radimrehurek.com/gensim/corpora/dictionary.html) :

In [15]:
corpus = [ dictionary.doc2bow(doc) for doc in docs]

## 3. Topic Modeling with LDA

Now it's time to train our LDA! To do this, we use the following parameters: 

- **corpus**: the bag-of-words representations of our documents
- **id2token**: the mapping of indexes to words
- **num_topics** : the number of topics the model should identify (let's set <font color = "red"><b>10</b></font>)
- **chunksize**: the number of documents the model sees on each update (let's set to <font color = "red"><b>1,000</b></font>)
- **passes**: the number of times we show the total corpus to the model during training (let's set to <font color = "red"><b>5</b></font>)
- **random_state**: we use a seed to ensure reproducibility (let's set to <font color = "red"><b>1</b></font>)

On a corpus of this size, training usually takes one or two minutes.

**Question 2 :**

In [16]:
from gensim.models import LdaModel

# TODO

## 4. Results and visualization

**Question 3 :** Let's see what the model has learned. To do this, let's display the ten most characteristic words for each topic.

In [17]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.023*"nucléaire" + 0.019*"énergies" + 0.018*"énergie" + 0.012*"france" + 0.011*"faut" + 0.011*"centrales" + 0.010*"renouvelables" + 0.009*"production" + 0.008*"électricité" + 0.007*"nucléaires"

***********
* topic 2 *
***********
2 : 0.039*"écologique" + 0.038*"transition" + 0.023*"transition_écologique" + 0.022*"faire" + 0.019*"faut" + 0.012*"écologie" + 0.008*"changer" + 0.008*"citoyens" + 0.007*"politique" + 0.007*"monde"

***********
* topic 3 *
***********
3 : 0.038*"transports" + 0.031*"transport" + 0.018*"routes" + 0.018*"camions" + 0.014*"commun" + 0.012*"vitesse" + 0.012*"développer" + 0.010*"ferroutage" + 0.009*"train" + 0.008*"pollution"

***********
* topic 4 *
***********
4 : 0.012*"économie" + 0.011*"voir" + 0.009*"système" + 0.009*"entreprises" + 0.008*"sanitaires" + 0.008*"déchets" + 0.008*"environnement" + 0.007*"environnemental" + 0.007*"respect" + 0.007*"informer"

***********
* topic 5 *
***********
5 : 0.009*"pouvoir" + 0.0

Another way to observe topics is to **visualize** them. This can be done with the library [pyLDAvis](https://github.com/bmabey/pyLDAvis). PyLDAvis will show us how popular the topics are in our corpus, how similar the topics are, and which words are most important for that topic. Note that it is important to set ``sort_topics = False`` on the call to pyLDAvis. If you don't, the topics will be sorted differently than in Gensim. This may take a few minutes to load.

**Question 5 :**

In [ ]:
import pyLDAvis.gensim
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

Finally, let's look at the topics that the model recognizes in some of the individual documents. Here we see how LDA tends to assign a high probability to a small number of topics for each document, making its results highly interpretable.

In [28]:
# Nous en affichons que 8
i = 0
for (text, doc) in zip(texts[:8], docs[:8]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

***********
* doc 1   *
***********
Multiplier les centrales géothermiques
[(1, 0.7745871)]

***********
* doc 2   *
***********
Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040.  Avec les progrès de la communication dans ce village mondial, chaque individu, du fin fond de l’Asie au fin fond de l’Afrique, en passant par les « quartiers » et les « campagnes » de notre pays, aspire à vivre – et on ne peu l’en blâmer – comme les moins mal lotis de nos concitoyens (logement, nourriture, biens de consommation, déplacement,etc.).  Voilà la mère de tous les problèmes. Si tel est bien le cas, la solution à tous les problèmes (stabilisation de la croissance démographique, partage des richesses, partage des terr

Many collections of unstructured text are not accompanied by labels. Topic models such as LDA are a useful technique for discovering the most important topics in these documents. **Gensim** facilitates learning about these topics and **pyLDAvis** presents the results in a visually appealing way. Together, they form a powerful toolkit for better understanding what's inside large document sets and for exploring subsets of related texts. While these results are often already quite revealing, it is also possible to use them as a starting point, for example, for a labeling exercise for supervised text classification. In sum, thematic models should be in every data scientist's toolbox as a very quick way to gain insight into large document collections.